In [ ]:
# !pip install tensorflow==1.13.2

ERROR: Could not find a version that satisfies the requirement tensorflow==1.13.2 (from versions: 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.19.0rc0)
ERROR: No matching distribution found for tensorflow==1.13.2


In [ ]:
# !pip install tensorflow

^C


  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached markdown_it_py-3.0.0-py3-non

In [1]:
import numpy as np # linear algebra
from tqdm.notebook import tqdm
import re, os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import tensorflow as tf

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk

import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

In [2]:
DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'

In [3]:
df = pd.read_csv("QTL_text.csv")

# rename category column as label
df.rename(columns={'Category': 'label'}, inplace=True) 

# combine title and abstract into one column called text
df['text'] = df['Title'] + ' ' + df['Abstract']


# only keep label and text columns
df = df[['label', 'text']]

In [4]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

# Text cleaning function
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"[^\w\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text)  # Remove extra whitespace
    text = " ".join([stemmer.stem(word) for word in text.split() if word not in stop_words])  # Remove stopwords and stem
    return text


df["Cleaned Text"] = df["text"].apply(clean_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rohai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
test_ratio = 0.2
train_ind, test_ind = train_test_split(df.index, test_size=test_ratio, shuffle=True, random_state=3)

train_df = df.loc[train_ind,:]
test_df = df.loc[test_ind,:]

In [ ]:
# Random oversampling of minority class (label 1)
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(train_df[["Cleaned Text"]], train_df["label"])

train_df = pd.DataFrame(X_resampled, columns=["Cleaned Text"])
train_df["label"] = y_resampled

In [6]:
# Undersampling the majority class (label 0) to balance the dataset

min_train_count = train_df["label"].value_counts().min()
train_df = pd.concat([group.sample(min_train_count) for name, group in train_df.groupby("label")])

In [7]:
from datasets import Dataset

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df[["Cleaned Text", "label"]])

In [8]:
split_dataset = dataset.train_test_split(test_size=0.2, seed=3)

# Access train and test splits
train_texts = split_dataset['train']
test_texts = split_dataset['test']

In [9]:
from transformers import AutoTokenizer

# Initialize tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Cleaned Text"], padding="max_length", truncation=True, max_length=512)

# Apply tokenization
train_dataset = train_texts.map(tokenize_function, batched=True)
test_dataset = test_texts.map(tokenize_function, batched=True)


# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/9022 [00:00<?, ? examples/s]

Map:   0%|          | 0/2256 [00:00<?, ? examples/s]

In [10]:
train_text = train_df["Cleaned Text"].values
train_labels = train_df["label"].values

val_text = test_df["Cleaned Text"].values
val_labels = test_df["label"].values

In [11]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2, output_attentions=False, output_hidden_states=True)
model.to("cuda")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
batch_size = 16

train_token_ids = []
train_attention_masks = []
for description in train_text:
  encoding_dict = tokenizer.encode_plus(
    description,
      add_special_tokens=True,
      max_length=512,
      padding='max_length',  
      truncation=True,       
      return_attention_mask=True,
      return_tensors="pt"
  )
  train_token_ids.append(encoding_dict["input_ids"]) 
  train_attention_masks.append(encoding_dict["attention_mask"])

train_token_ids = torch.cat(train_token_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_labels = torch.tensor(train_labels)

val_token_ids = []
val_attention_masks = []
for description in val_text:
  encoding_dict = tokenizer.encode_plus(
    description,
    add_special_tokens=True,
    max_length=512,
    padding='max_length',  
    truncation=True,       
    return_attention_mask=True,
    return_tensors="pt"
  )
  val_token_ids.append(encoding_dict["input_ids"]) 
  val_attention_masks.append(encoding_dict["attention_mask"])

val_token_ids = torch.cat(val_token_ids, dim=0)
val_attention_masks = torch.cat(val_attention_masks, dim=0)
val_labels = torch.tensor(val_labels)

train_set = TensorDataset(train_token_ids, train_attention_masks, train_labels)
val_set = TensorDataset(val_token_ids, val_attention_masks, val_labels)

# Prepare DataLoader
train_dataloader = DataLoader(train_set, sampler=RandomSampler(train_set), batch_size=batch_size)
val_dataloader = DataLoader(val_set, sampler=SequentialSampler(val_set), batch_size=batch_size)

In [13]:
from sklearn.metrics import f1_score

def f1_score_func(predictions, labels):
    preds_flat = np.argmax(predictions, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average="weighted")

In [14]:
epochs = 12
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader)*epochs)

c:\Users\rohai\anaconda3\envs\new_env\lib\site-packages\transformers\optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
def evaluate(val_dataloader):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in val_dataloader:
        
        batch = tuple(b.to("cuda") for b in batch)
        
        inputs = {"input_ids":      batch[0],
                  "attention_mask": batch[1],
                  "labels":         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs["labels"].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(val_dataloader) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [16]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(train_dataloader, desc="Epoch {:1d}".format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to("cuda") for b in batch)
        
        inputs = {"input_ids":      batch[0],
                  "attention_mask": batch[1],
                  "labels":         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({"training_loss": "{:.3f}".format(loss.item()/len(batch))})
         
    # save model to directory, if it doesn't exist create it
    if not os.path.exists("./content/"):
        os.makedirs("./content/")
   
    torch.save(model.state_dict(), f"./bertfiles/finetuned_BERT_epoch_{epoch}.model")
        
    tqdm.write(f"\nEpoch {epoch}")
    
    loss_train_avg = loss_train_total/len(train_dataloader)            
    tqdm.write(f"Training loss: {loss_train_avg}")
    
    val_loss, predictions, true_vals = evaluate(val_dataloader)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f"Validation loss: {val_loss}")
    tqdm.write(f"F1 Score (Weighted): {val_f1}")

  0%|          | 0/12 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/101 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB (GPU 0; 4.00 GiB total capacity; 10.61 GiB already allocated; 0 bytes free; 10.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [24]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels=2, 
    state_dict=torch.load("./content/finetuned_BERT_epoch_5.model", map_location="cuda")
)
model.to("cuda")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [2]:
print("Starting preprocessing...")
check_df = pd.read_csv("test_unlabeled.csv")
check_df["text"] = check_df["Title"] + " " + check_df["Abstract"]
check_df["Cleaned Text"] = check_df["text"].apply(clean_text)

check_text = check_df["Cleaned Text"].values

check_token_ids = []
check_attention_masks = []
for description in check_text:
  encoding_dict = tokenizer.encode_plus(description, add_special_tokens=True, max_length=256 ,  pad_to_max_length=True, return_attention_mask=True, return_tensors="pt")
  check_token_ids.append(encoding_dict["input_ids"]) 
  check_attention_masks.append(encoding_dict["attention_mask"])

check_token_ids = torch.cat(check_token_ids, dim=0)
check_attention_masks = torch.cat(check_attention_masks, dim=0)

check_set = TensorDataset(check_token_ids, check_attention_masks)
check_dataloader = DataLoader(check_set, sampler=SequentialSampler(check_set), batch_size=batch_size)

print("Finished preprocessing, now loading model...")

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels=2, 
    state_dict=torch.load("./bertfiles/checkpoints/finetuned_BERT_epoch_12.model", map_location="cuda")
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set the model to evaluation mode
model.eval()

print("Model loaded, now predicting...")

predictions = []

# No gradient calculation is needed during inference
with torch.no_grad():
    progress = tqdm(check_dataloader, desc="Predicting")
    for batch in progress:
        batch = tuple(b.to("cuda") for b in batch)

        inputs = {"input_ids": batch[0], "attention_mask": batch[1]}

        # Get model outputs
        outputs = model(**inputs)
        logits = outputs.logits  # Extract logits

        probs = F.softmax(logits, dim=1)

        # Convert logits to predicted class
        preds = torch.argmax(probs, dim=1).cpu().numpy()
        predictions.extend(preds)

# Convert lists to NumPy arrays
predictions = np.array(predictions)

print("Prediction complete!")

# predictions = np.concatenate(predictions, axis=0)
# predictions = np.argmax(predictions, axis=1).flatten()

check_df["Category"] = predictions
check_df.to_csv("submission.csv", index=False)


Starting preprocessing...


NameError: name 'clean_text' is not defined

In [6]:
from transformers import pipeline

ner_pipeline = pipeline("ner", model="dmis-lab/biobert-v1.1", tokenizer="dmis-lab/biobert-v1.1")

text = "In a previous study, QTL for carcass composition and meat quality were identified in a commercial finisher cross. The main objective of the current study was to confirm and fine map the QTL on SSC4 and SSC11 by genotyping an increased number  of individuals and markers and to analyze the data using a combined linkage and linkage disequilibrium analysis method."
entities = ner_pipeline(text)

print(entities)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


[{'entity': 'LABEL_1', 'score': 0.5241997, 'index': 1, 'word': 'In', 'start': 0, 'end': 2}, {'entity': 'LABEL_0', 'score': 0.5435446, 'index': 2, 'word': 'a', 'start': 3, 'end': 4}, {'entity': 'LABEL_1', 'score': 0.5095409, 'index': 3, 'word': 'previous', 'start': 5, 'end': 13}, {'entity': 'LABEL_0', 'score': 0.5702159, 'index': 4, 'word': 'study', 'start': 14, 'end': 19}, {'entity': 'LABEL_0', 'score': 0.5313562, 'index': 5, 'word': ',', 'start': 19, 'end': 20}, {'entity': 'LABEL_0', 'score': 0.5712199, 'index': 6, 'word': 'Q', 'start': 21, 'end': 22}, {'entity': 'LABEL_0', 'score': 0.62011474, 'index': 7, 'word': '##TL', 'start': 22, 'end': 24}, {'entity': 'LABEL_0', 'score': 0.57298166, 'index': 8, 'word': 'for', 'start': 25, 'end': 28}, {'entity': 'LABEL_0', 'score': 0.64287645, 'index': 9, 'word': 'car', 'start': 29, 'end': 32}, {'entity': 'LABEL_0', 'score': 0.6728945, 'index': 10, 'word': '##cas', 'start': 32, 'end': 35}, {'entity': 'LABEL_0', 'score': 0.6390939, 'index': 11, 'w